In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pymongo
from datetime import datetime
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from scipy.spatial.distance import cosine


client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["AttendanceSystem"]
userCOLLECTION = db["Users"]
CollectionATTANDANCE = db["Attendance"]


base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

def faceEmba(facePIC):
    facePIC = cv2.resize(facePIC, (224, 224))
    facePIC = img_to_array(facePIC)
    facePIC = preprocess_input(facePIC)
    facePIC = np.expand_dims(facePIC, axis=0)
    embedding = base_model.predict(facePIC)[0]
    return embedding


def userdata(name, facePIC):
    embedding = faceEmba(facePIC)
    userENTRED = {"name": name, "embedding": embedding.tolist()}  
    userCOLLECTION.insert_one(userENTRED)
    print(f"{name}'s face embedding!")


def regSystem(faceDATA):
    users = userCOLLECTION.find({})
    bestMATCH = None
    BESTsimilarity = 0
    
    for user in users:
        STOREDemb = np.array(user["embedding"])
        similarity = 1 - cosine(faceDATA, STOREDemb)  
        print(f"{user['name']}: Similarity = {similarity:.4f}") 
        
        if similarity > 0.35 and similarity > BESTsimilarity: 
            bestMATCH = user["name"]
            BESTsimilarity = similarity
    
    return bestMATCH


def mark_attendance(name):
    MarkedDATE = str(datetime.today().date())
    alreadyMARKED = CollectionATTANDANCE.find_one({"name": name, "date": MarkedDATE})
    
    if not alreadyMARKED:
        attandanceENTRY = {
            "name": name,
            "date": MarkedDATE,
            "time": str(datetime.now().strftime("%H:%M:%S"))
        }
        CollectionATTANDANCE.insert_one(attandanceENTRY)
        print(f"Attendance Marked: {name}")
    else:
        print(f"{name}'s attendance already marked today.")


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        faceDATA = faceEmba(face)
        name = regSystem(faceDATA)

        if name:
            mark_attendance(name)
            text = f"Recognized: {name}"
            color = (0, 255, 0)
        else:
            text = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow("Face Recognition Attendance", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()